In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score, roc_auc_score, average_precision_score, recall_score,
    precision_score, f1_score, matthews_corrcoef
)
from flaml import AutoML
import matplotlib.pyplot as plt
import joblib
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedGroupKFold

/n/groups/patel/adithya/scenv/lib/python3.10/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [2]:
#log_dir_path = "/n/groups/patel/adithya/Syn18_Log_Dir_Total_on_cell/"

In [3]:
exp_type = 'maximal'
cell_type = 'Mic'

# Load the data
metadata = pd.read_parquet('/home/adm808/New_CellMetadataSyn1848517.parquet')
print("Metadata is loaded")

# Process APOE genotype as categorical -- Hot encoding of apoe_genotype
metadata = pd.get_dummies(metadata, columns=["apoe_genotype"])
apoe_genotype_columns = [col for col in metadata.columns if col.startswith("apoe_genotype_")]


# Stratified Shuffle Split based on `sample_id`to split metadata
# Define Alzheimer's or control status directly based on `age_first_ad_dx`
metadata = metadata.copy()
metadata['alzheimers_or_control'] = metadata['age_first_ad_dx'].notnull().astype(int)

# Extract unique sample IDs and their associated Alzheimer's/control status -- drop duplicates
sample_summary = metadata[['sample', 'alzheimers_or_control', 'msex']].drop_duplicates()

# I need to create a combined stratification variable
sample_summary['stratify_group'] = sample_summary['alzheimers_or_control'].astype(str) + "_" + sample_summary['msex'].astype(str)

# Perform stratified train-test split on `sample_id`, stratified by `alzheimers_or_control`
train_samples, test_samples = train_test_split(
    sample_summary['sample'],
    test_size=0.2,
    random_state=42,
    stratify=sample_summary['stratify_group']
)

# Filter metadata by train and test `sample_id`
train_metadata = metadata[metadata['sample'].isin(train_samples)]
test_metadata = metadata[metadata['sample'].isin(test_samples)]


# We only want to predict on one cell type but train the model on all cell types so we filter test_metadata
test_metadata = test_metadata[test_metadata['broad.cell.type'] == cell_type]


print(f"Number of cases in training: {sum(train_metadata['alzheimers_or_control'])}")
print(f"Number of cases in test: {sum(test_metadata['alzheimers_or_control'])}")



Metadata is loaded
Number of cases in training: 21958
Number of cases in test: 131


In [5]:
print(f"Number of cases in test: {sum(test_metadata['alzheimers_or_control'])}")

Number of cases in test: 131


In [6]:
print(len(train_metadata))
print(len(test_metadata))

55677
569


In [7]:
print(len(train_samples))

38


In [8]:
sample_summary.head()

,sample,alzheimers_or_control,msex,stratify_group
0,1,0,1.0,0_1.0
312,2,0,1.0,0_1.0
432,3,0,1.0,0_1.0
847,4,1,1.0,1_1.0
1694,5,0,0.0,0_0.0


In [9]:
train_metadata_check = train_metadata[train_metadata['broad.cell.type'] == cell_type]
print(f"Number of cases in training: {sum(train_metadata_check['alzheimers_or_control'])}")
print(train_metadata_check)

Number of cases in training: 529
                       TAG    projid     tsne1      tsne2  pre.cluster  \
62407   AACTTTCAGGATGGTC.1  11409232  4.511704 -48.516566           13   
62408   ACGGGCTCAATCAGAA.1  11409232  5.457969 -52.755649           13   
62409   ACTTACTGTGAAATCA.1  11409232  7.256031  58.172625           13   
62410   AGCGGTCAGATGTTAG.1  11409232  6.462830 -52.549235           13   
62411   AGTGAGGGTGGTAACG.1  11409232  4.764862 -48.622495           13   
...                    ...       ...       ...        ...          ...   
64322  CCGTTCACAGCGAACA.48  11302830  5.086001 -53.544812           13   
64323  CCTTCGATCAAACCGT.48  11302830  4.228903 -52.145368           13   
64324  CGATGTAAGGGTTTCT.48  11302830  7.317785 -39.182029           13   
64325  TAAGAGATCGTGGGAA.48  11302830  2.416547  54.937268           13   
64326  TGGCTGGAGTGAATTG.48  11302830  2.664527 -52.428202           13   

      broad.cell.type Subcluster  msex     age_first_ad_dx  braaksc  ...  \
62

In [10]:
print(f"Train samples: {train_samples}")

Train samples: 6665     10
312       2
23908    32
23006    30
16553    22
18385    25
15890    21
21667    28
20506    27
26370    36
0         1
15464    20
4242      8
13149    17
23269    31
13781    18
34757    48
29160    41
6066      9
14739    19
30285    42
10319    14
22176    29
26958    37
33230    46
28129    39
19284    26
847       4
12668    16
17355    23
27488    38
30903    43
3629      7
31956    44
26267    35
8478     11
28537    40
1694      5
Name: sample, dtype: int64


In [11]:
print(f"\nTest samples: {test_samples}")


Test samples: 432       3
17757    24
34185    47
25663    34
25077    33
2962      6
11132    15
32504    45
9310     13
8770     12
Name: sample, dtype: int64


In [12]:
print(test_metadata['sample'].unique())

[ 3  6 12 13 15 24 33 34 45 47]


In [13]:
all_cell_type = pd.concat([train_metadata_check, test_metadata])

In [14]:
all_cell_type['sample'].value_counts()

sample
15    111
5     105
3     102
28     86
24     84
44     73
7      73
10     72
11     69
6      65
26     62
16     60
45     55
47     48
25     47
4      47
38     46
21     45
17     44
34     44
19     43
29     39
22     34
2      33
13     32
32     31
14     31
18     28
36     26
40     25
41     25
20     25
39     25
46     25
1      21
30     20
42     18
33     18
23     14
43     13
31     12
9      11
12     10
48      6
37      5
27      5
8       4
35      3
Name: count, dtype: int64

In [15]:
# Function to select and drop missing genes
def select_missing_genes(filtered_matrix):
    mean_threshold = 1
    missingness_threshold = 95

    mean_gene_expression = filtered_matrix.mean(axis=0)
    missingness = (filtered_matrix == 0).sum(axis=0) / filtered_matrix.shape[0] * 100
    null_expression = (missingness > missingness_threshold) & (mean_gene_expression < mean_threshold)
    genes_to_drop = filtered_matrix.columns[null_expression].tolist()

    return genes_to_drop

# Load and transpose gene expression matrices
gene_matrix = pd.read_parquet('/home/adm808/NormalizedCellMatrixSyn18485175.parquet').T
print("Gene matrix is loaded")
print(gene_matrix.iloc[:, :5].head())

# Defining training and testing matrices
train_matrix = gene_matrix.loc[train_metadata['TAG']]
test_matrix = gene_matrix.loc[test_metadata['TAG']]

print("Printing dimensionality of X_train and X_test initallly")
print(train_matrix.shape)
print(test_matrix.shape)

# Filter missing genes
train_matrix_filtered = train_matrix.drop(select_missing_genes(train_matrix), axis=1)
test_matrix_filtered = test_matrix.drop(select_missing_genes(test_matrix), axis=1)

# Merge the train and test matrices with their respective metadata files

train_data = train_matrix_filtered.merge(
    train_metadata[['TAG', 'msex', 'sample', 'broad.cell.type', 'alzheimers_or_control', 'age_death', 'educ', 'cts_mmse30_lv', 'pmi'] + apoe_genotype_columns],
    left_index=True,
    right_on='TAG',
    how='inner'
).set_index('TAG')

test_data = test_matrix_filtered.merge(
    test_metadata[['TAG', 'msex', 'sample', 'broad.cell.type', 'alzheimers_or_control', 'age_death', 'educ', 'cts_mmse30_lv', 'pmi'] + apoe_genotype_columns],
    left_index=True,
    right_on='TAG',
    how='inner'
).set_index('TAG')


# Clean column names for model compatibility
train_data.columns = train_data.columns.str.replace(r'[^A-Za-z0-9_]+', '', regex=True)
test_data.columns = test_data.columns.str.replace(r'[^A-Za-z0-9_]+', '', regex=True)

# Ensure common genes are used between training and testing sets
common_genes = train_data.columns.intersection(test_data.columns)
X_train = train_data[common_genes]
X_test = test_data[common_genes]

# Drop the alzheimers or control column from the dataset
X_train = X_train.drop(columns=['alzheimers_or_control'])
X_test = X_test.drop(columns=['alzheimers_or_control'])

# Map original column names to cleaned names for later interpretability
original_columns = common_genes  # Use common genes after filtering
cleaned_columns = original_columns.str.replace(r'[^A-Za-z0-9_]+', '', regex=True)
column_mapping = dict(zip(cleaned_columns, original_columns))

# Define the target variable
y_train = train_data['alzheimers_or_control']
y_test = test_data['alzheimers_or_control']

print("Printing dimensionality of X_train and X_test post filtering and merging")

print(X_train.shape)
print(X_test.shape)



Gene matrix is loaded
                    FO538757.2  SAMD11     NOC2L  KLHL17  PLEKHN1
AAACGGGAGATCCCGC.1    0.000000     0.0  0.000000     0.0      0.0
AAATGCCTCCAATGGT.1    6.919779     0.0  0.000000     0.0      0.0
AACCATGTCAGTGCAT.1   11.637401     0.0  0.000000     0.0      0.0
AACCATGTCTGTACGA.1    0.000000     0.0  6.974242     0.0      0.0
AACCGCGTCCGCATAA.1    7.831354     0.0  0.000000     0.0      0.0
Printing dimensionality of X_train and X_test initallly
(55677, 17926)
(569, 17926)
Printing dimensionality of X_train and X_test post filtering and merging
(55677, 2440)
(569, 2440)


In [16]:
#########################################################################
# Trying a new method of creating cross validation folds:
from sklearn.model_selection import StratifiedGroupKFold
import numpy as np

def generate_valid_folds(X, y, groups, n_splits=10, max_retries=100):
    """
    Generate valid folds for StratifiedGroupKFold to ensure no fold has only one class.
    Retries until valid folds are created.
    """
    retries = 0
    while retries < max_retries:
        retries += 1
        valid_folds = True
        stratified_group_kfold = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=np.random.randint(1000))
        
        # Generate folds
        folds = list(stratified_group_kfold.split(X, y, groups))

        # Check for class balance in validation folds
        for fold, (train_idx, val_idx) in enumerate(folds):
            train_y, val_y = y.iloc[train_idx], y.iloc[val_idx]
            if len(val_y.unique()) < 2:  # Check if validation set has both classes
                print(f"Retry {retries}: Fold {fold + 1} has only one class. Retrying...")
                valid_folds = False
                break

        if valid_folds:
            print(f"Valid folds generated after {retries} retries.")
            return folds  # Return valid folds

    raise ValueError("Unable to generate valid folds after maximum retries.")


# This seciton is throwing an error, seems like flaml has no argument to create custom folds, must follow approach taken below:
# # Generate valid folds
# valid_folds = generate_valid_folds(
    # X_train,  # Feature matrix
#     y_train,  # Target variable
#     groups=train_metadata['sample'],  # Group variable
#     n_splits=10,
#     max_retries=100
# )

#cell_log_dir = os.path.join(log_dir_path, cell_type)

# Create the directory if it doesn’t exist
#os.makedirs(cell_log_dir, exist_ok=True)


# For task one I am training on all cell types, but testing only on one specific cell type. Therefore, I will subset just the testing sets for cell type:

# Dropping samples from the dataset
X_train = X_train.drop(columns=['sample'])
X_test = X_test.drop(columns=['sample'])


In [17]:
X_train.shape, X_test.shape

((55677, 2439), (569, 2439))